In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd
import dask


In [2]:
%matplotlib inline
%autosave 60

Autosaving every 60 seconds


In [3]:
cluster = PBSCluster(
    cores=1,
    memory="120GB",
    project='pangeo',
    processes=1,
    walltime='12:00:00',
    local_directory='$TMPDIR')
cluster

PBSCluster('tcp://10.120.41.63:33395', workers=0, threads=0, memory=0 B)

In [4]:
cluster.scale(40)

In [5]:
client = Client(cluster) # scheduler_file="/home/ad/briolf/scheduler.json")
client

Client Scheduler: tcp://10.120.41.63:33395 Dashboard: http://10.120.41.63:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
print(len(client.scheduler_info()["workers"]))

0


In [7]:
!qstat -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 30:
        break
    time.sleep(1)
print(nb_workers)

30


In [8]:
import zarr
import netCDF4
import pytide
import xarray as xr
import os
import numpy as np

In [20]:
%%time
root = "/work/ALT/odatis/eNATL60/BLBT02/gridT-2D/"
files = [
    os.path.join(root, item) for item in os.listdir(root) if item.endswith(".nc")]

drop_vars = [
    'nav_lat',
    'nav_lon',
    'somxl010',
    'sosaline',
    'sosstsst']

# these are variables I want to drop while running `open_mfdataset` but then add back later
extra_coord_vars = ['y', 'x']
extra_coord_vars = []

chunks = dict(time_counter=1)

open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks, decode_cf=True, concat_dim="time_counter") #, combine='nested')
ds = xr.open_mfdataset(files, combine='nested',parallel=True, **open_kwargs)


CPU times: user 11.7 s, sys: 2.67 s, total: 14.4 s
Wall time: 48 s


In [21]:
ds.sossheig.shape

(11688, 4729, 8354)

In [22]:
START_DATE = np.datetime64('2009-07-01')
END_DATE = np.datetime64('2010-06-30')
time_series=ds['time_counter']
period = (time_series >= START_DATE) & (time_series <= END_DATE)
time=time_series[period]
ssh=ds.sossheig[period,:,0:4177]
t=time.values

In [28]:
time_series

,Array,Chunk
Bytes,93.50 kB,8 B
Shape,"(11688,)","(1,)"
Count,23863 Tasks,11688 Chunks
Type,datetime64[ns],numpy.ndarray


In [12]:
%%time
wt = pytide.WaveTable()
wt = pytide.WaveTable(["M2", "S2", "N2", "O1", "K1"])
f, vu = wt.compute_nodal_modulations(t)

CPU times: user 8.53 ms, sys: 0 ns, total: 8.53 ms
Wall time: 7.83 ms


In [13]:
def dask_array_rechunk(da, axis=0):
    """Search for the optimal block cutting without modifying the axis 'axis'
    in order to optimize its access in memory."""
    nblocks = 1
    
    def calculate_chuncks_size(chunks, size):
        result = np.array(chunks).prod() * size
        return result / (1000**2)
       
    while True:
        chunks = []
        div = int(np.sqrt(nblocks))
        for index, item in enumerate(da.chunks):
            chunks.append(np.array(item).sum() * (div if index == axis else 1))
        chunks = tuple(item // div for index, item in enumerate(chunks))
        chuncks_size = calculate_chuncks_size(chunks, da.dtype.itemsize)
        if chuncks_size > 100 and chuncks_size < 150:
            return chunks
        nblocks += 1

In [14]:
def _apply_along_axis(arr, func1d, func1d_axis, func1d_args, func1d_kwargs):
    """Wrap apply_along_axis"""
    return np.apply_along_axis(func1d, func1d_axis, arr, *func1d_args,
                                  **func1d_kwargs)


def apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """Apply the harmonic analysis to 1-D slices along the given axis."""
    arr = dask.array.core.asarray(arr)

    # Validate and normalize axis.
    arr.shape[axis]
    axis = len(arr.shape[:axis])

    # Rechunk so that analyze is applied over the full axis.
    arr = arr.rechunk(arr.chunks[:axis] + (arr.shape[axis:axis + 1], ) +
                      arr.chunks[axis + 1:])

    # Test out some data with the function.
    test_data = np.ones(args[0].shape[1], dtype=arr.dtype)
    test_result = np.array(func1d(test_data, *args, **kwargs))

    # Map analyze over the data to get the result
    # Adds other axes as needed.
    result = arr.map_blocks(
        _apply_along_axis,
        name=dask.utils.funcname(func1d) + '-along-axis',
        dtype=test_result.dtype,
        chunks=(arr.chunks[:axis] + test_result.shape + arr.chunks[axis + 1:]),
        drop_axis=axis,
        new_axis=list(range(axis, axis + test_result.ndim, 1)),
        func1d=func1d,
        func1d_axis=axis,
        func1d_args=args,
        func1d_kwargs=kwargs,
    )

    return result

In [15]:
ssh_rechunk = ssh.chunk(dask_array_rechunk(ssh))
future = apply_along_axis(pytide.WaveTable.harmonic_analysis, 0, ssh_rechunk,
                          *(f, vu))


In [16]:
len(future.dask)

460313

In [ ]:
%%time
analysis = future.compute()

In [ ]:
%%time
nt=len(time)
nwaves, ni, nj = analysis.shape
for k in np.arange(len(time)):
    if not os.path.exists('/work/ALT/odatis/eNATL60/outputs/pytide/eNATL60miX1-BLBT02_tide_y2009m07-y2010m06_k'+str(k)+'.nc'):
        ttide = wt.tide_from_mapping(
        time[k].astype('datetime64[s]').astype('float64'),
        analysis.reshape(nwaves, ni*nj)).reshape(ni, nj)
        ttide_da=xr.DataArray(ttide,dims={'y','x'},name='tide')
        ttide_da.to_netcdf(path='/work/ALT/odatis/eNATL60/outputs/pytide/eNATL60miX1-BLBT02_tide_y2009m07-y2010m06_k'+str(k)+'.nc',engine='scipy')

In [ ]:
cluster.close()